In [1]:
import pandas as pd
import numpy as np

from datetime import timedelta

import seaborn as sns
import matplotlib.pyplot as plt
import time
import math

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_excel('../../data/market_data.xlsx')

In [4]:
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

In [5]:
vic = data[['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)', 
                 'Regions VIC Trading Total Intermittent Generation (MW)', 
                 'Regions VIC Operational Demand (MW)']]

In [6]:
# Since the first date is at 00:00:00, the first period should be 48
period = [48]
x = 1
while x < len(vic):
    for i in range(48):
        period.append(i+1)
        x += 1
        
vic.insert(1, 'Period', period)
vic.drop([0], axis=0)

,Time (UTC+10),Period,Regions VIC Trading Price ($/MWh),Regions VIC Trading Total Intermittent Generation (MW),Regions VIC Operational Demand (MW)
1,2018-01-01 00:30:00,1,92.46,131.68,4398
2,2018-01-01 01:00:00,2,87.62,119.98,4238
3,2018-01-01 01:30:00,3,73.08,123.86,4112
4,2018-01-01 02:00:00,4,70.18,132.72,3956
5,2018-01-01 02:30:00,5,67.43,120.73,3833
...,...,...,...,...,...
63452,2021-08-14 22:00:00,44,49.93,182.52,5492
63453,2021-08-14 22:30:00,45,62.86,206.23,5344
63454,2021-08-14 23:00:00,46,32.26,215.20,5204
63455,2021-08-14 23:30:00,47,25.10,226.95,5268


In [25]:
def first_cycle(spot_price):
    """ Returns first indexes of the first periods for the first cycles """
    
    max_price = 0
    min_price = 999999

    for i in range(48-6):
        """ Finds which 6 periods have the most sum and least sum 
            by going through 1 to 6, 2 to 7 and so on """

        curr = spot_price.iloc[i:i+6 ,0].sum()

        if curr < min_price:
            min_price = curr
            min_i = i  # Stores the first index of the max period

        if curr > max_price:
            max_price = curr
            max_i = i  # Stores the first index of the min period
            
            
            
    min_, max_ = store_index(min_i, max_i)
                
    return min_, max_

In [8]:
def sec_cycle(spot_price, min_index, max_index):
    """ Returns first indexes of the first periods for the second cycles """
    
    remaining = list(spot_price.index)
    sec_min_price = 999999
    sec_max_price = 0
    to_remove = min_index + max_index
    
    # remove all periods after max
    remaining = remaining[:remaining.index(to_remove[-1])+1]
    
    for index in to_remove:
        remaining.remove(index)

    for i in range(len(remaining) - 6):

        # make sure the next six indexes are increment of 1
        if remaining[i] == (remaining[i+5] - 5):

            curr_sum = spot_price.iloc[remaining[i]:remaining[i]+6 ,0].sum()

            if curr_sum < sec_min_price:
                sec_min_price = curr_sum
                sec_min_i = remaining[i]  # Stores the first index of the second max period

            if curr_sum > sec_max_price:

                sec_max_price = curr_sum
                sec_max_i = remaining[i]  # Stores the first index of the second min period
                
    min_, max_ = store_index(sec_min_i, sec_max_i)
    
    min_index += min_
    max_index += max_
                
    return min_index, max_index

In [9]:
def store_index(index1, index2):
    """ Store the rest of the max and min price indexes """
    list1 = []
    list2 = []
    for i in range(6):
        list1.append(index1 + i)
        list2.append(index2 + i)
        
    return list1, list2

In [20]:
def algorithm2(ori_df):
    """ Finds optimal charge and discharge period from the mean """

    monday_df = ori_df[ori_df['day'] == 1]
    tuesday_df = ori_df[ori_df['day'] == 2]
    wednesday_df = ori_df[ori_df['day'] == 3]
    thursday_df = ori_df[ori_df['day'] == 4]
    friday_df = ori_df[ori_df['day'] == 5]
    saturday_df = ori_df[ori_df['day'] == 6]
    sunday_df = ori_df[ori_df['day'] == 7]

    
    days_df = [monday_df, tuesday_df, wednesday_df, thursday_df, friday_df, saturday_df, sunday_df]
    
    discharge_period = []
    charge_period = []
    for i in range(len(days_df)):
        spot_price = days_df[i].groupby(['Period'])[[PRICE]].mean()

        # First cycle
        min_, max_ = first_cycle(spot_price)

        # Second cycle
        # Comment line 11 if only want one cycle
        min_, max_ = sec_cycle(spot_price, min_, max_)
        
        # the charge and discharge period are fixed in Algorithm 2, +1 to get their periods
        min_ = list(np.asarray(min_) + 1)
        max_ = list(np.asarray(max_) + 1)
        
        discharge_period.append(max_)
        charge_period.append(min_)
    
    return charge_period, discharge_period

In [27]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, 'Period', PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    df['revenue'] = 0
    df['decision'] = 0
    
    # I removed the first row because first row is 00:00:00, 
    # which is the last period from the previous day
    #df = df.drop([0], axis=0)
    
    # convert to datetime
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    
    df['day'] = df[TIME].dt.weekday + 1
    
    
    return df

In [12]:
def get_day(day):
    
    if (day == 1):
        return 0
    elif (day == 2):
        return 1
    elif (day == 3):
        return 2
    elif (day == 4):
        return 3
    elif (day == 5):
        return 4
    elif (day == 6):
        return 5
    elif (day == 7):
        return 6

In [13]:
def find_all(ori_df):
    """ Returns a completed dataframe """
    """ This is the main function, calling this function will automatically run all other functions """
    
    start = time.time()
    
    df = create_df(ori_df)
    charge_period, discharge_period = algorithm2(df)
    #print(charge_period[0])
    #print(discharge_period[0])
    df = df.reset_index(drop=True)
    df.index += 1
    
    for i in list(df.index):

        period = df.at[i, "Period"]
        price = df.at[i, PRICE]

        """ Find Opening Cap """

        if i != 1:
            df.at[i,"opening"] = df.at[i-1,"closing"]

        opening_cap = math.ceil(df.at[i, "opening"])




        """ Find raw_power """
        day = get_day(df.at[i, 'day'])
        
        if period in charge_period[day]:
                df.at[i, "raw_power"] = -math.floor(min(POWER,(CAPACITY-opening_cap)/(CHARGE_EFF/100)*2))

        elif period in discharge_period[day]:
            df.at[i, "raw_power"] = math.floor(min(POWER,opening_cap/(DISCHARGE_EFF/100)*2))

        rawPower = df.at[i, "raw_power"]




        """ Find dispatch """
        if rawPower < 0:
            eff = 1

        else:
            eff = DISCHARGE_EFF / 100

        df.at[i,"dispatch"] = math.ceil(rawPower / 2 * eff)
        dispatch = df.at[i, "dispatch"]



        """ Find Closing Cap """
        if dispatch < 0:
            eff = CHARGE_EFF / 100

        else:
            eff = 100 / DISCHARGE_EFF

        df.at[i,"closing"] = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))



        """ Find revenue """
        if dispatch < 0:
            factor = 1/MLF

        else:
            factor = MLF

        df.at[i,"revenue"] = math.ceil(price * dispatch * factor)
        
    
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    end = time.time()
    print("Time Complexity for running the entire Algorithm 2: {time_taken}s".format(time_taken = end-start))
    
    return df

In [28]:
alg = find_all(vic)

[4, 5, 6, 7, 8, 9, 22, 23, 24, 25, 26, 27]
[34, 35, 36, 37, 38, 39, 13, 14, 15, 16, 17, 18]
Total revenue in the dataset: 68493692
Total days in the dataset: 1322.0208333333333
Revenue per day: 51809.84313787289
Time Complexity for running the entire Algorithm 2: 4.200225114822388s


In [ ]:
# vic['month'] = vic['Time (UTC+10)'].dt.month

In [ ]:
# SUMMER = [12,1,2]
# AUTUMN = [3,4,5]
# WINTER = [6,7,8]
# SPRING = [9,10,11]



# for i in list(vic.index):
#     if ((vic.at[i, 'month'] in SUMMER)):
#         vic.at[i,'season'] = 0
#     elif ((vic.at[i,'month'] in AUTUMN)):
#         vic.at[i,'season'] = 1
#     elif ((vic.at[i,'month'] in WINTER)):
#         vic.at[i,'season'] = 2
#     else:
#         vic.at[i,'season'] = 3

In [ ]:
# summer_vic = vic.loc[vic['season'] == 0]
# autumn_vic = vic.loc[vic['season'] == 1]
# winter_vic = vic.loc[vic['season'] == 2]
# spring_vic = vic.loc[vic['season'] == 3]